In [1]:
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams.update({'font.size': 18})
matplotlib.rcParams['lines.linewidth'] = 0.5
matplotlib.rcParams['lines.markeredgewidth'] = 0.5
matplotlib.rcParams['lines.markersize'] = 8 

In [8]:
import numpy as np
from numba import njit, prange, set_num_threads
from itertools import product

set_num_threads(15)

In [4]:
from ns2d import (
    EulerIntegrator,
    FiniteDifferenceDiscretizer,
    FiniteDifferenceUpwindDiscretizer,
    FiniteVolumeDiscretizer,
    GaussSeidelSolver,
    JacobiSolver,
    PredictorCorrectorIntegrator,
    RK4Integrator
)

In [7]:
nx, ny = 41, 41
dx, dy = 2.0 / (nx - 1), 2.0 / (ny - 1)
dt = 0.001
nu = 1e-2
num_steps = 500

x = np.linspace(0, 2, nx)
y = np.linspace(0, 2, ny)
X, Y = np.meshgrid(x, y)

rhs_discretizers = [
    FiniteDifferenceDiscretizer(),
    FiniteDifferenceUpwindDiscretizer(),
    FiniteVolumeDiscretizer()
]
predictors = [
    EulerIntegrator(),
    PredictorCorrectorIntegrator,
    RK4Integrator
]
all_combinations = list(product(rhs_discretizers, predictors))

NameError: name 'product' is not defined

# Results for Jacobi Solver

In [ ]:
# for discretizer, predictor in combinations:
#     rhs_name = discretizer.__class__.__name__
#     lhs_name = predictor.__class__.__name__
    
#     print(f"\nTesting combination:")
#     print(f" - Discretizer: {rhs_name}")
#     print(f" - Predictor: {lhs_name}")
    
#     try:
#         # Run actual simulation
#         # ...
        
#         # Print or store the result
#         # results[(rhs_name, lhs_name)] = result
        
#     except Exception as e:
#         print(f"Error occurred: {e}")

In [6]:
# plt.figure(figsize=(11, 7))
# plt.contourf(X, Y, solver.p, alpha=0.5)
# plt.colorbar(label='Pressure')
# plt.contour(X, Y, solver.p, colors='black', linestyles='solid')
# plt.quiver(X[::2, ::2], Y[::2, ::2], solver.u[::2, ::2], solver.v[::2, ::2], color='r')
# plt.xlabel('X')
# plt.ylabel('Y')
# plt.tight_layout()
# plt.show()

# Results for Gauss-Seidel Solver